In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
path_in = "/Users/sofia/Documents/Repos/skytruth_30x30/data/raw"
path_out = "/Users/sofia/Documents/Repos/skytruth_30x30/data/processed"

In [9]:
eez = gpd.read_file(path_in + "/World_EEZ_v11_20191118/eez_v11.shp")
eez.head(1)

,MRGID,GEONAME,MRGID_TER1,POL_TYPE,MRGID_SOV1,TERRITORY1,ISO_TER1,SOVEREIGN1,MRGID_TER2,MRGID_SOV2,...,ISO_SOV1,ISO_SOV2,ISO_SOV3,UN_SOV1,UN_SOV2,UN_SOV3,UN_TER1,UN_TER2,UN_TER3,geometry
0,8444.0,American Samoa Exclusive Economic Zone,8670.0,200NM,2204.0,American Samoa,ASM,United States,0.0,0.0,...,USA,NaN,NaN,840,NaN,NaN,16.0,NaN,NaN,"POLYGON ((-166.64112 -17.55527, -166.64194 -17..."


In [4]:
eez.columns

Index(['MRGID', 'GEONAME', 'MRGID_TER1', 'POL_TYPE', 'MRGID_SOV1',
       'TERRITORY1', 'ISO_TER1', 'SOVEREIGN1', 'MRGID_TER2', 'MRGID_SOV2',
       'TERRITORY2', 'ISO_TER2', 'SOVEREIGN2', 'MRGID_TER3', 'MRGID_SOV3',
       'TERRITORY3', 'ISO_TER3', 'SOVEREIGN3', 'X_1', 'Y_1', 'MRGID_EEZ',
       'AREA_KM2', 'ISO_SOV1', 'ISO_SOV2', 'ISO_SOV3', 'UN_SOV1', 'UN_SOV2',
       'UN_SOV3', 'UN_TER1', 'UN_TER2', 'UN_TER3', 'geometry'],
      dtype='object')

In [5]:
eez.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [45]:
# Check if eez geometries are valid
sum(eez.geometry.is_valid)

In [10]:
eez = eez.to_crs('ESRI:54009')
eez['AREA_KM2']= eez.geometry.area/ 1000000
eez.head(5)

,MRGID,GEONAME,MRGID_TER1,POL_TYPE,MRGID_SOV1,TERRITORY1,ISO_TER1,SOVEREIGN1,MRGID_TER2,MRGID_SOV2,...,ISO_SOV1,ISO_SOV2,ISO_SOV3,UN_SOV1,UN_SOV2,UN_SOV3,UN_TER1,UN_TER2,UN_TER3,geometry
0,8444.0,American Samoa Exclusive Economic Zone,8670.0,200NM,2204.0,American Samoa,ASM,United States,0.0,0.0,...,USA,NaN,NaN,840,NaN,NaN,16.0,NaN,NaN,"POLYGON ((-16216412.543 -2157569.856, -1621650..."
1,8379.0,Ascension Exclusive Economic Zone,8620.0,200NM,2208.0,Ascension,SHN,United Kingdom,0.0,0.0,...,GBR,NaN,NaN,826,NaN,NaN,654.0,NaN,NaN,"POLYGON ((-1089355.142 -974062.004, -1089348.4..."
2,8446.0,Cook Islands Exclusive Economic Zone,8672.0,200NM,2227.0,Cook Islands,COK,New Zealand,0.0,0.0,...,NZL,NaN,NaN,554,NaN,NaN,184.0,NaN,NaN,"POLYGON ((-15912583.852 -716733.193, -15813064..."
3,8389.0,Overlapping claim Falkland / Malvinas Islands:...,8623.0,Overlapping claim,2208.0,Falkland / Malvinas Islands,FLK,United Kingdom,8623.0,2149.0,...,GBR,ARG,NaN,826,32.0,NaN,238.0,238.0,NaN,"POLYGON ((-4061728.309 -6509190.466, -4443979...."
4,8440.0,French Polynesian Exclusive Economic Zone,8656.0,200NM,17.0,French Polynesia,PYF,France,0.0,0.0,...,FRA,NaN,NaN,250,NaN,NaN,258.0,NaN,NaN,"MULTIPOLYGON (((-13543804.433 -974376.651, -13..."


In [11]:
eez.to_file(path_out + "/administrative/eez_area_mollweide.shp", driver="ESRI Shapefile")

In [13]:
# Reproject to 4626
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shp -proj EPSG:4326 -o force /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shp

Allocating 16 GB of heap memory
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shp
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shx
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.dbf
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.prj


In [15]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'GL',
        'region_name': 'Global',
        'country_iso_3s': []
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_name']

In [16]:
eez['REGIONS'] = eez['ISO_SOV1'].map(country_to_region)
eez.head(2)

,MRGID,GEONAME,MRGID_TER1,POL_TYPE,MRGID_SOV1,TERRITORY1,ISO_TER1,SOVEREIGN1,MRGID_TER2,MRGID_SOV2,...,ISO_SOV2,ISO_SOV3,UN_SOV1,UN_SOV2,UN_SOV3,UN_TER1,UN_TER2,UN_TER3,geometry,REGIONS
0,8444.0,American Samoa Exclusive Economic Zone,8670.0,200NM,2204.0,American Samoa,ASM,United States,0.0,0.0,...,NaN,NaN,840,NaN,NaN,16.0,NaN,NaN,"POLYGON ((-16216412.543 -2157569.856, -1621650...",North America
1,8379.0,Ascension Exclusive Economic Zone,8620.0,200NM,2208.0,Ascension,SHN,United Kingdom,0.0,0.0,...,NaN,NaN,826,NaN,NaN,654.0,NaN,NaN,"POLYGON ((-1089355.142 -974062.004, -1089348.4...",Europe


In [19]:
eez.to_file(path_out + "/administrative/eez_mollweide_regions.shp", driver="ESRI Shapefile")

In [18]:
eez['REGIONS'].unique()

array(['North America', 'Europe', 'Asia & Pacific',
       'Latin America & Caribbean', 'Africa', 'West Asia', nan],
      dtype=object)

In [20]:
# Dissolve by relevant fields: REGIONS
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_mollweide_regions.shp -dissolve2 fields=REGIONS -explode -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp

Allocating 16 GB of heap memory
[dissolve2] Removed 127,740 / 218,614 slivers using 0.033+ sqkm variable threshold
[dissolve2] Dissolved 281 features into 7 features
[explode] Exploded 7 features into 83 features
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shx
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.dbf
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.prj


In [21]:
regions = gpd.read_file(path_out + "/administrative/eez_regions.shp", driver="ESRI Shapefile")
regions.head(2)

,REGIONS,geometry
0,North America,"POLYGON ((-16216412.543 -2157569.856, -1621650..."
1,North America,"POLYGON ((-15875617.974 972834.674, -15887321...."


In [22]:
regions.crs

<Projected CRS: ESRI:54009>
Name: World_Mollweide
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Mollweide
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
regions['REGIONS'].unique()

array(['North America', 'Europe', 'Asia & Pacific',
       'Latin America & Caribbean', 'Africa', 'West Asia', nan],
      dtype=object)

In [24]:
regions['REGIONS'][regions['REGIONS'].isna()] = 'Antartica'

/var/folders/98/0pdnjc5s29x2pnzl293pw7hr0000gn/T/ipykernel_27590/1686611470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regions['REGIONS'][regions['REGIONS'].isna()] = 'Antartic'


In [28]:
# Calculate area of each region
regions['AREA_KM2']= regions.geometry.area/ 1000000

In [29]:
regions.to_file(path_out + "/administrative/eez_regions.shp", driver="ESRI Shapefile")

In [30]:
# Reproject to 4626
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp -proj EPSG:4326 -o force /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp

Allocating 16 GB of heap memory
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shx
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.dbf
[o] Wrote /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.prj


In [32]:
regions['REGIONS'].unique()

array(['North America', 'Europe', 'Asia & Pacific',
       'Latin America & Caribbean', 'Africa', 'West Asia', 'Antarctica'],
      dtype=object)